In [1]:
"""
Create the generator module
"""
import re
import numpy as np
import torch.nn as nn
import torch

In [16]:


from importlib import reload


class GeneratorNet(nn.Module):
    """ take a tensor with accuracies for each of the classes
    and predict a probability distribution for the classes that
    will maximize the loss of the classifier
    """

    # pylint: disable=too-many-instance-attributes
    # pylint: disable=too-many-arguments
    def __init__(
        self,  
        n_classes, 
        n_hidden_layers,
        n_hidden_features,
    ):
        super().__init__()

        # initialize some values
        self.n_classes = n_classes
        self.n_hidden_layers = n_hidden_layers
        self.n_hidden_features = n_hidden_features
        self.sequence = []

        ####################################################
        ########             layers               ##########
        ####################################################
        
        # dense layers
        # initialize the first layer
        n_features_prev = self.n_classes
        n_features_out = self.n_hidden_features
        # stack linears
        for i in range(n_hidden_layers):
            setattr(
                self,
                "linear" + str(i),
                nn.Linear(
                    in_features=n_features_prev,
                    out_features=n_features_out,
                ),
            )
            n_features_prev = n_features_out
            # add the output layer
            print(i)
            if i+2 == self.n_hidden_layers:
                n_features_out = self.n_classes

        # activation layers
        self.relu = nn.LeakyReLU(0.2)
        self.softmax = nn.Softmax(dim=1)
        self.define_sequence()

    def define_sequence(self):
        "set sequence to make it easier for the forward pass"
        
        for name, seq_module in self.named_modules():
            number = []

            # set linear layer
            if isinstance(seq_module, nn.Linear):
                self.sequence.append(seq_module)
                self.sequence.append(self.relu)
                number = re.findall(r"\d+", name)

        # remove the last relu and change to softmax
        self.sequence.pop(-1)
        self.sequence.append(self.softmax)

    def forward(self, tensor):
        "forward pass of the MGC classifier"

        # iterate through the layers
        for layer in self.sequence:
            # apply the layer
            tensor = layer(tensor)
        return tensor
    
test_probabilities = torch.rand([2,10])
test_generator = GeneratorNet(
    n_classes = 10,
    n_hidden_layers = 3,
    n_hidden_features = 5,
)

print(test_generator)

test_generator(test_probabilities)

0
1
2
GeneratorNet(
  (linear0): Linear(in_features=10, out_features=5, bias=True)
  (linear1): Linear(in_features=5, out_features=5, bias=True)
  (linear2): Linear(in_features=5, out_features=10, bias=True)
  (relu): LeakyReLU(negative_slope=0.2)
  (softmax): Softmax(dim=1)
)


tensor([[0.1118, 0.1154, 0.1077, 0.1190, 0.0702, 0.0690, 0.0666, 0.1529, 0.1217,
         0.0659],
        [0.1138, 0.1171, 0.1107, 0.1179, 0.0686, 0.0702, 0.0631, 0.1544, 0.1168,
         0.0674]], grad_fn=<SoftmaxBackward>)